In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random
import warnings
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import matplotlib_venn
import graph_tool.all as gt
from tqdm import tqdm
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

from llm_ol.dataset.data_model import load_graph
from llm_ol.utils.nx_to_gt import nx_to_gt
from llm_ol.experiments.post_processing import hp_search, post_process, PostProcessHP
from llm_ol.eval.graph_metrics import edge_precision, edge_recall, edge_f1
from metadata import query, query_multiple

warnings.filterwarnings("ignore")

In [ ]:
# Some utilities

fig_dir = Path("out", "graphs")


def display_graph(G: nx.Graph, layout: str = "dot"):
    relabel_map = {}
    for n, data in G.nodes(data=True):
        relabel_map[n] = data.get("title", n)
    G = nx.relabel_nodes(G, relabel_map)
    A = nx.nx_agraph.to_agraph(G)
    A.node_attr.update(fontname="Helvetica", fontsize=10, shape="plaintext")
    A.graph_attr.update(ratio="compress", ranksep=0.25)
    A.edge_attr.update(arrowsize=0.5)
    A.layout(layout)
    display(A)


def nth_level_nodes(G: nx.Graph, n: int):
    return nx.descendants_at_distance(G, G.graph["root"], n)


def nth_level_edges(G: nx.Graph, n: int):
    distances = nx.single_source_shortest_path_length(G, G.graph["root"], cutoff=n)
    return {(u, v) for u, v in G.edges() if distances.get(u, None) == n}


def title(G, n):
    return G.nodes[n].get("title", n)

## Plot subgraph induced by paths

In [ ]:
exp = query(exp="finetune", step=30000, reweighted=True)
G = load_graph(exp.eval_output)
G = post_process(G, PostProcessHP(edge_percentile=0.975, merge_nodes_by_lemma=False))

In [ ]:
path_cutoff = 4
node = random.choice(list(G.nodes()))

G_sub = nx.DiGraph()
for path in nx.all_simple_paths(G, G.graph["root"], node, cutoff=path_cutoff):
    for u, v in zip(path[:-1], path[1:]):
        G_sub.add_edge(title(G, u), title(G, v))

display_graph(G_sub)

## Plot the union and intersection of nodes & edges in the train, eval and test graphs

In [ ]:
exp = query(exp="hearst")
G_train = load_graph(exp.train_input)
G_eval = load_graph(exp.eval_ground_truth)
G_test = load_graph(exp.test_ground_truth)

fig, axs = plt.subplots(2, 3, figsize=(12, 8))

for level, ax in enumerate(axs[0], start=1):
    set1 = nth_level_nodes(G_train, level)
    set2 = nth_level_nodes(G_eval, level)
    set3 = nth_level_nodes(G_test, level)
    matplotlib_venn.venn3([set1, set2, set3], ["Train", "Eval", "Test"], ax=ax)
    ax.set_title(f"Level {level} nodes")

for level, ax in enumerate(axs[1], start=0):
    set1 = nth_level_edges(G_train, level)
    set2 = nth_level_edges(G_eval, level)
    set3 = nth_level_edges(G_test, level)
    matplotlib_venn.venn3([set1, set2, set3], ["Train", "Eval", "Test"], ax=ax)
    ax.set_title(f"Level {level} edges")

# fig.savefig(fig_dir / "wiki_train_eval_test_split.png", dpi=300)

## Plot the nodes & edges coverage by only considering paths of length n

In [ ]:
def node_and_edge_coverage(G: nx.Graph, n: int):
    G_gt, nx_to_gt_map, gt_to_nx_map = nx_to_gt(G)

    nodes_with_pages = {
        node for node, data in G.nodes(data=True) if len(data["pages"]) > 0
    }

    nodes_covered = set()
    edges_covered = set()
    for node in tqdm(nodes_with_pages):
        for path in gt.all_paths(
            G_gt,
            source=nx_to_gt_map[G.graph["root"]],
            target=nx_to_gt_map[node],
            cutoff=n,
        ):
            edges_covered |= {
                (gt_to_nx_map[u], gt_to_nx_map[v]) for u, v in zip(path[:-1], path[1:])
            }
            nodes_covered |= {gt_to_nx_map[v] for v in path}

    assert nodes_covered.issubset(set(G.nodes()))
    assert edges_covered.issubset(set(G.edges()))
    return (
        len(nodes_covered) / G.number_of_nodes(),
        len(edges_covered) / G.number_of_edges(),
    )


fig, ax = plt.subplots(1, 1, figsize=(5, 3))
xs = np.arange(6)
ys = np.array([node_and_edge_coverage(G_train, n) for n in xs])
ax.plot(xs, ys[:, 0], label="Nodes coverage")
ax.plot(xs, ys[:, 1], label="Edges coverage")
fig.legend(loc="upper left")

# fig.savefig(fig_dir / "wiki_test_coverage.png", dpi=300)

## Motif analysis

In [ ]:
def regular_polygon_layout(g: gt.Graph, n: int, r: float = 1.0):
    pos = g.new_vertex_property("vector<double>")
    for i, v in enumerate(g.vertices()):
        pos[v] = (
            r * np.cos(2 * np.pi * i / n - np.pi / 2),
            r * np.sin(2 * np.pi * i / n - np.pi / 2),
        )
    return pos


def count_motifs(Gs: list[nx.Graph], n: int = 3):
    motifs_list, counts_list = zip(*[gt.motifs(nx_to_gt(G)[0], n) for G in Gs])
    all_motifs = []
    all_idx_to_idx = []
    for motifs in motifs_list:
        idx_to_idx = {}  # idx in all_motifs -> idx in motifs
        for i, motif in enumerate(motifs):
            for j, existing_motif in enumerate(all_motifs):
                if gt.isomorphism(motif, existing_motif):
                    idx_to_idx[j] = i
                    break
            else:
                all_motifs.append(motif)
                idx_to_idx[len(all_motifs) - 1] = i
        all_idx_to_idx.append(idx_to_idx)

    all_counts = []
    for idx_to_idx, counts in zip(all_idx_to_idx, counts_list):
        all_counts.append([])
        for j in range(len(all_motifs)):
            all_counts[-1].append(counts[idx_to_idx[j]] if j in idx_to_idx else 0)

    return all_motifs, all_counts


motifs, counts = count_motifs([G_train, G_eval, G_test])

# sort by the sum of counts
counts = np.array(counts)
order = np.argsort(counts.sum(axis=0))[::-1]
counts = counts[:, order]
motifs = [motifs[i] for i in order]

# only keep the top n motifs
n = 5
counts = counts[:, :n]
motifs = motifs[:n]

# normalize the counts
counts = counts / counts.sum(axis=1)[:, None]

df = pd.DataFrame(
    {
        "count": counts.reshape(-1),
        "motif": np.tile(np.arange(len(motifs)), counts.shape[0]),
        "graph": np.repeat(["Train", "Eval", "Test"], len(motifs)),
    }
)

fig, ax = plt.subplots(figsize=(8, 4))

sns.barplot(data=df, x="motif", y="count", hue="graph", ax=ax)
ax.set(xticklabels=[], xlabel="", ylabel="Fraction of motifs")

# draw the motif graph as the x labels
res = 5
r = 0.7
pad = 0.7
for motif, xticklabel in zip(motifs, ax.get_xticklabels()):
    gt.graph_draw(
        motif,
        pos=regular_polygon_layout(motif, motif.num_vertices(), r),
        vertex_fill_color="black",
        # vertex_size=5,
        edge_color="black",
        output_size=(30 * res, 30 * res),
        output="/tmp/motif.png",
        ink_scale=0.6,
        fit_view=(-r - pad, -r - pad, 2 * (r + pad), 2 * (r + pad)),
    )
    im = plt.imread("/tmp/motif.png")
    ib = OffsetImage(im, zoom=1 / res, snap=True, resample=True)
    ib.image.axes = ax
    ab = AnnotationBbox(
        ib,
        xticklabel.get_position(),
        frameon=False,
        box_alignment=(0.5, 1.1),
    )
    ax.add_artist(ab)

## AP vs training step

In [ ]:
exps = query_multiple(exp="finetune", version=2)

assert len({exp.eval_ground_truth for exp in exps}) == 1
G_true = load_graph(exps[0].eval_ground_truth)


def prec_recall_curve(thresholds, G_pred, G_true):
    precisions = []
    recalls = []

    for edge_percentile in tqdm(thresholds):
        G_pruned = post_process(
            G_pred,
            PostProcessHP(
                edge_percentile=edge_percentile,
                merge_nodes_by_lemma=False,
                prune_unconnected_nodes=True,
            ),
        )
        prec = edge_precision(G_pruned, G_true)
        rec = edge_recall(G_pruned, G_true)
        precisions.append(prec)
        recalls.append(rec)

    return np.array(precisions), np.array(recalls)


data = []
for exp in exps:
    G = load_graph(exp.eval_output)
    thresholds = 1 - np.geomspace(1 / G.number_of_edges(), 1, 11)
    precisions, recalls = prec_recall_curve(thresholds, G, G_true)
    data.append(
        {
            "step": exp.step,
            "reweighted": exp.reweighted,
            "precisions": precisions,
            "recalls": recalls,
        }
    )

In [ ]:
def agg_ap(group):
    ap = np.trapz(group["precisions"], group["recalls"])
    return pd.Series({"ap": ap})


df = pd.concat([pd.DataFrame(d) for d in data])
df["f1"] = 2 * df["precisions"] * df["recalls"] / (df["precisions"] + df["recalls"])

fig, axs = plt.subplots(1, 3, figsize=(12, 3))

sns.lineplot(
    data=df, x="recalls", y="precisions", hue="step", style="reweighted", ax=axs[0]
)
df_ap = df.groupby(["step", "reweighted"]).apply(agg_ap)
sns.lineplot(data=df_ap, x="step", y="ap", style="reweighted", ax=axs[1])
df_f1 = df.groupby(["step", "reweighted"]).agg({"f1": "max"})
sns.lineplot(data=df_f1, x="step", y="f1", style="reweighted", ax=axs[2])

fig.tight_layout()

In [ ]:
def agg_ap(group):
    ap = np.trapz(group["precisions"], group["recalls"])
    return pd.Series({"ap": ap})


df = pd.concat([pd.DataFrame(d) for d in data])
df["f1"] = 2 * df["precisions"] * df["recalls"] / (df["precisions"] + df["recalls"])

fig, axs = plt.subplots(1, 3, figsize=(12, 3))

sns.lineplot(
    data=df, x="recalls", y="precisions", hue="step", style="reweighted", ax=axs[0]
)
df_ap = df.groupby(["step", "reweighted"]).apply(agg_ap)
sns.lineplot(data=df_ap, x="step", y="ap", style="reweighted", ax=axs[1])
df_f1 = df.groupby(["step", "reweighted"]).agg({"f1": "max"})
sns.lineplot(data=df_f1, x="step", y="f1", style="reweighted", ax=axs[2])

fig.tight_layout()

## Finetune overfitting

In [ ]:
exps = query_multiple(exp="finetune")

assert len({exp.eval_ground_truth for exp in exps}) == 1
G_true = load_graph(exps[0].eval_ground_truth)

n_levels = 4


data = []
for exp in exps:
    G = load_graph(exp.eval_output)
    for level in range(n_levels):
        target_edges = [
            (title(G_true, u), title(G_true, v))
            for u, v in nth_level_edges(G_true, level)
        ]
        weights = [
            G.edges[u, v]["weight"] if G.has_edge(u, v) else 0 for u, v in target_edges
        ]
        data.append(
            {
                "step": exp.step,
                "reweighted": exp.reweighted,
                "level": level,
                "weight": weights,
            }
        )

df = pd.concat([pd.DataFrame(d) for d in data])

fig, axs = plt.subplots(1, n_levels, figsize=(4 * n_levels, 3))

for level, ax in enumerate(axs):
    sns.lineplot(
        data=df[df["level"] == level],
        x="step",
        y="weight",
        hue="reweighted",
        ax=ax,
        errorbar=None,
    )
    ax.set(title=f"Mean weight of level {level} edges")

fig.tight_layout()
fig.savefig(fig_dir / "finetune_detailed_edge_weights_change.png", dpi=300)